# Traveling Wave Detection with MC-SPEt algorithm 

## Required imports

In [1]:
import h5py
import numpy as np
import pandas as pd
from alphashape import alphashape
from Detection_functions import MCA, circular_mask, preprocessing

## Choose dataset and associated parameters for detection algorithm

In [4]:
condition='awake' #between anesthesia and awake
session='evoked' #between evokedU, evokedD and spontaneous

In [5]:
eps=17
MinPts=32
theta=.40
sign_level=1e-6

## Data loading

In [6]:
original_data_directory=r'/media/mathis/Data/Mathis/Original Data/'

In [7]:
if condition=='awake':
    fs = 100 #Hz
    pixel_spacing = 0.0566 #mm
    if session=='evoked':
        bks_evoked_path = original_data_directory+'gaussian_pos4_ziggy_210908.npy'
        Data = np.load(bks_evoked_path)
    elif session=='spontaneous':
        bks_blank_path = original_data_directory+'blank_ziggy_210908.npy'
        Data = np.load(bks_blank_path)
    mean = np.nanmean(Data)
    nan_loc = np.isnan(Data)
    Data[nan_loc] = mean
    N_trials, Nt, Nx, Ny = Data.shape
    mask = circular_mask((Nx,Ny), center=(220,190), radius=180) #160 should be more relevant

if condition=='anesthesia':
    fs=110 #Hz
    pixel_spacing = 0.0575 #mm
    if session=='evokedD':
        anesth_evokedD_path = original_data_directory+'signal_240306_gaussianD.mat'
        file_anesth_evokedD = h5py.File(anesth_evokedD_path)['signal']
        Data = np.array(file_anesth_evokedD)
    elif session=='evokedU':
        anesth_evokedU_path = original_data_directory+'signal_240306_gaussianU.mat'
        file_anesth_evokedU = h5py.File(anesth_evokedU_path)['signal']
        Data = np.array(file_anesth_evokedU)
    elif session=='spontaneous':
        anesth_blank_path = original_data_directory+'signal_240306_blank.mat' #change it to adapt to your directories
        file_anesth_blank = h5py.File(anesth_blank_path)['signal']
        Data = np.array(file_anesth_blank)
    N_trials, Nt, Nx, Ny = Data.shape
    mask = circular_mask((Nx,Ny), center=(125,140), radius=130) #155 should be more relevant


In [8]:
180*pixel_spacing

10.187999999999999

In [10]:
np.sum(mask)*pixel_spacing**2

323.29686807999997

## Run data-collecting algorithms

In [6]:
Waves=[]
for n in range(N_trials):
    print(f'trial {n}/{N_trials}')
    preprocessed_trial = preprocessing(trial=Data[n], border_mask=mask, fs=fs, fc=25, threshold=99)
    moving_clusters = MCA(Trial=preprocessed_trial, epsilon=eps, min_pts=MinPts, Theta=theta, significance_level=sign_level)
    for index, mc in enumerate(moving_clusters): #labelling of the waves
        mc['wave label'] = (n,index) #(trial number, wave number)
        Waves.append(mc)
    print(f'{len(moving_clusters)} waves detected')

trial 0/20
0 waves detected
trial 1/20
2 waves detected
trial 2/20
3 waves detected
trial 3/20
3 waves detected
trial 4/20
4 waves detected
trial 5/20
3 waves detected
trial 6/20
4 waves detected
trial 7/20
2 waves detected
trial 8/20
0 waves detected
trial 9/20
2 waves detected
trial 10/20
3 waves detected
trial 11/20
3 waves detected
trial 12/20
1 waves detected
trial 13/20
3 waves detected
trial 14/20
1 waves detected
trial 15/20
2 waves detected
trial 16/20
3 waves detected
trial 17/20
2 waves detected
trial 18/20
1 waves detected
trial 19/20
3 waves detected


In [7]:
len(Waves)

45

In [8]:
for wave in Waves:
    print((wave['wave label'],wave['trace']))

((1, 0), [33, 34, 35, 36, 37, 38])
((1, 1), [40, 41, 42, 43, 44, 45, 46, 47])
((2, 0), [24, 25, 26, 27, 28])
((2, 1), [29, 30, 31, 32, 33, 34])
((2, 2), [91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114])
((3, 0), [2, 3, 4, 5, 6, 7, 8])
((3, 1), [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
((3, 2), [98, 99, 100, 101, 102, 103])
((4, 0), [32, 33, 34, 35])
((4, 1), [115, 116, 117, 118, 119])
((4, 2), [132, 133, 134, 135])
((4, 3), [138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148])
((5, 0), [35, 36, 37, 38, 39])
((5, 1), [55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67])
((5, 2), [68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88])
((6, 0), [37, 38, 39, 40, 41])
((6, 1), [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43])
((6, 2), [138, 139, 140, 141, 142, 143])
((6, 3), [145, 146, 147, 148])
((7, 0), [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
((7, 1), [6, 7, 8, 9, 10, 11, 12, 13]

## Features selection and data frame Construction

In [9]:
Df=[]
lab = []
for ind,wave in enumerate(Waves):
    n=len(wave['trace'])
    center,surface = [],[]
    for i in range(n):
        M = np.mean(wave['points'][i],axis=0 )
        center.append(M)
        polyg=alphashape(wave['points'][i], .1)
        surface.append(polyg.area)
    Centre=np.array(center)
    Dist = np.linalg.norm(np.diff(Centre,axis=0), axis=1)
    Instant_speed = np.append(Dist*fs*pixel_spacing/1000, np.nan)
    vx,vy=wave['Speed']*fs*pixel_spacing/1000
    Velocity_coordinates = np.append([vx, vy, np.linalg.norm([vx,vy]), np.arctan2(vy,vx)],np.full(n-4,np.nan)) #vx,vy,r,theta+nan to fill
    
    data = {'Wave #':np.full(n,ind),'Trial #':np.full(n,wave['wave label'][0]),'index':range(n),
            'Frames': wave['trace'],'Time (ms)': np.array(wave['trace'])/fs*1000,'x_center':Centre[:,0], 'y_center':Centre[:,1], 
            'Center speed (m/s)': Instant_speed, 'Velocity': Velocity_coordinates,'Span (mm2)': np.array(surface)*pixel_spacing**2}
    df = pd.DataFrame(data)
    Df.append(df)
Final_df = pd.concat(Df, axis=0, ignore_index=True)

In [10]:
Final_df.head(15)

,Wave #,Trial #,index,Frames,Time (ms),x_center,y_center,Center speed (m/s),Velocity,Span (mm2)
0,0,1,0,33,300.000000,221.539604,114.064356,0.112910,-0.011684,14.992191
1,0,1,1,34,309.090909,215.054441,97.432665,0.141367,0.042462,27.729519
2,0,1,2,35,318.181818,206.187234,117.948936,0.147338,0.044040,19.391156
3,0,1,3,36,327.272727,208.756757,141.101351,0.102112,1.839305,11.869438
4,0,1,4,37,336.363636,209.769231,124.989011,0.052197,NaN,7.420878
5,0,1,5,38,345.454545,214.932039,131.427184,NaN,NaN,7.969716
6,1,1,0,40,363.636364,213.662602,136.479675,0.120634,-0.000142,20.173084
7,1,1,1,41,372.727273,213.114815,155.544444,0.022880,-0.016924,21.117019
8,1,1,2,42,381.818182,210.750988,152.806324,0.159244,0.016925,19.035734
9,1,1,3,43,390.909091,210.786802,127.629442,0.166476,-1.579159,16.179134


## Data Frame saving

In [11]:
filename=f'{condition.capitalize()}_{session}_bks.xlsx'
print(filename)

Anesthesia_evokedD_bks.xlsx


In [12]:
Final_df.to_excel(filename)